In [1]:
from firedrake import *
from firedrake_adjoint import *
from firedrake.ml.pytorch import *
from physics_driven_ml.models import EncoderDecoder
from physics_driven_ml.utils import ModelConfig

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance
/home/jbt20/hybrid_fixed/firedrake/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n = 50
mesh = RectangleMesh(n, n, 1, 1, name="mesh")
V = FunctionSpace(mesh, "CG", 1)
V1 = VectorFunctionSpace(mesh, "CG", 1)

In [3]:
config = ModelConfig(input_shape=V1.dim())
model = EncoderDecoder(config)
model.double()

EncoderDecoder(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_encoder): Linear(in_features=5202, out_features=2601, bias=True)
  (linear_decoder): Linear(in_features=2601, out_features=5202, bias=True)
)

In [4]:
def assemble_L2_error(x, x_exact):
    """Assemble L2-loss"""
    return assemble(0.5 * (x - x_exact) ** 2 * dx)

In [5]:
u_in = Function(V1).assign(0.5)
u_true = Function(V1).assign(1.0)

u_P = to_torch(u_in)
u_true_P = to_torch(u_true)

f_P = model(u_P)

In [6]:
u_obs = Function(V1)
u = Function(V1)

In [7]:
with set_working_tape() as tape:
        # Define PyTorch operator for computing the L2-loss (for computing κ -> 0.5 * ||κ - κ_exact||^{2}_{L2})
        F = ReducedFunctional(assemble_L2_error(u, u_obs), [Control(u), Control(u_obs)])
        H = torch_operator(F)

In [8]:
loss_P = H(f_P,u_true_P)

In [9]:
# -- Check backpropagation API -- #
loss_P.backward()

ValueError: ('Mismatched shapes in operands %s and %s', (2,), (1,))